# Diff Im Dolphot Package Tutorial

In [ ]:
import diff_im_dolphot as did
import os

File paths need to be defined to various directories and programs.

|       Property      | Description                                                   |
|:-------------------:|:-------------------------------------------------------------:|
|        imroot       | Base directory where files and subdirectories will be created |
|     orig_im_loc     | Location of the directory holding all the science images      |
|        im_loc       | Working directory of science images for Diff Im Dolphot       |
|      ref_image      | Location of refrence image                                    |
|     dolphot_path    | Location of bin folder in Dolphot installation                |
| sn_ra_me, sn_dec_me | Transient RA and Dec                                          |
|       sexpath       | Path to Source Extractor excecutable                          |

In [ ]:
imroot = os.getcwd()
orig_im_loc = f'{imroot}/image_backup'
im_loc = f'{imroot}/Images'
ref_image = f'{imroot}/ref/registration.fits'
dolphot_path = '/home/thomas/HST_Diff/dolphot2.0/bin'
sn_ra_me, sn_dec_me = '11:49:35.658', '+22:24:48.03'
sexpath = '/usr/bin/source-extractor'

Some Dolphot properties need to be defined prior running Diff Im Dolphot.

|       Property      | Description                                                   |
|:-------------------:|:-------------------------------------------------------------:|
|        raper        | Defines the size of the photometric aperture                  |
|         rchi        | Defines the size of the aperture in which the chi value is calculated|
|       rsky0         | Inner radius of sky annulus |
|       rsky1         | Outer radius of sky annulus |
|       rpsf          | Size of PSF used for star subtraction |
|     ACSpsfType      | Use of Anderson PSF cores |

The following Dolphot parameters are those recommended in the ACS Dolphot manual, for the exeption of using the Anderson PSF cores.

In [ ]:
dolphot_params = {
    'raper': 3,
    'rchi': 2.0,
    'rsky0': 15,
    'rsky1': 35,
    'rpsf': 10,
    'ACSpsfType': 1,
    }

First, a list of global constants needs to be defined and science images need to be copied from the original directory to a working directory.

In [ ]:
did.prep_dir(orig_im_loc,
             im_loc,
             ref_image,
             dolphot_path,
             imroot,
             sexpath,
             sn_ra_me,
             sn_dec_me,
             dolphot_params)

The science images need to be run though the AstroDrizzle/Tweakreg pipeline for alignment, cosmic ray detection, distortion correction, and stacking. After, Source-Extractor will then build a source catalog.

In [ ]:
did.mk_diff(threshold=10,
            fitgeometry='rscale',
            nclip=3,
            minobj=15,
            final_scale_coadd=0.03,
            ra_coadd=177.397192792,
            dec_coadd=22.3937313215,
            nx_coadd=5500,
            ny_coadd=5500)

The science images then need to be prepared for Dolphot processing. The masking, splitgroups, and calcsky scripts are called automatically.

In [ ]:
did.prep_files_for_dolphot('/dolphot_prepped',
                           r_in=15,
                           r_out=35,
                           step=4,
                           sig_low=2.25,
                           sig_high=2.00)

A Dolphot parameter file needs to be generated and then used to run the Dolphot script.

In [ ]:
did.dolphot_simultaneous()

In [ ]:
did.blot_back(r_in=15,
              r_out=35,
              step=4,
              sig_low=2.25,
              sig_high=2.00)

In [ ]:
objCoords = {
    'S1': [2635.27, 2978.30, 177.398231, 22.41111],
    'S2': [2692.26, 2997.02, 177.397718, 22.41111],
    'S3': [2730.87, 2967.26, 177.397370, 22.41111],
    'S4': [2682.07, 2925.37, 177.397810, 22.41111],
    'SX': [2426.58, 3106.85, 177.400112, 22.41111],
    }

did.dolphot_force(apermag=False, force_same_mag=False, psfphot=1, objCoords=objCoords)
did.dolphot_force(apermag=False, force_same_mag=True,  psfphot=1, objCoords=objCoords)
did.dolphot_force(apermag=False, force_same_mag=False, psfphot=2, objCoords=objCoords)
did.dolphot_force(apermag=False, force_same_mag=True,  psfphot=2, objCoords=objCoords)